<a href="https://colab.research.google.com/github/ijul17/homework/blob/main/Yulia%20Urazova/Legprom_last_data_Tfidf_model_with_2_fabrics_4_cols_reduced_with_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
from gensim import corpora,models,similarities
from gensim.utils import tokenize


In [ ]:
from google.colab import files
tables=files.upload()

Saving ТЗ для Датасетов доп фабрики.xlsx to ТЗ для Датасетов доп фабрики.xlsx


##Фабрики

In [ ]:
import pandas as pd
import numpy as np
xl = pd.ExcelFile('ТЗ для Датасетов доп фабрики.xlsx')
print(xl.sheet_names)
df_fabrics = xl.parse('Фабрики')
df_fabrics.head(3)


['Техзадания v020424', 'Фабрики', 'Сопоставление полей']


,ID пользователя,Регистрация,Логотип,Название,ИНН,Email,Телефон,Сайт,Имя контактного лица,Тип работ,...,Сырье,Нанесение,Оборудование,Штат,Упаковка,Условия оплаты,Дополнительные услуги,Сертификация,Загруженность,Наличии в реестре Минпромторга
0,8517,2024-03-26 11:00:44,https://tpktrade.ru/wp-content/uploads/2024/03...,"ООО """"ЗИТЕКС",9731087878,zitexgroup@mail.ru,+79260410006,https://poshivopt.com/index.html,Айнура,Производство,...,Давальческое,цифровая печать,Прямострочка,"Конструктор на аутсорсинге, 10 швей",NaN,"Аванс 50% / Расчет 50%, Безналичная оплата, Бе...",NaN,Нет,Необходимо согласование не менее чем за 14 дней,Нет
1,8408,2024-03-12 14:31:39,https://tpktrade.ru/wp-content/uploads/2024/03...,Lavender Magic,760414418671,lavendermagicshop@gmail.com,+79159727049,NaN,Полина,Производство,...,Закупает фабрика,dtf,Флэтлог,"конструктор на аутсорсе, 30 швей",NaN,"Аванс 50% / Расчет 50%, Безналичная оплата, Бе...",NaN,Нет,Необходимо согласование не менее чем за 14 дней,Нет
2,8360,2024-03-05 10:21:59,NaN,ООО Мой Стиль,0,mystyle579@mail.ru,NaN,NaN,NaN,Производство,...,Давальческое,Вышивка,Вышивальная машина,конструктор в штате 50 швей,NaN,"100"" аванс",NaN,Да,Всегда есть возможность принять небольшие заказы,Да


In [ ]:
df_fabrics.drop(['Регистрация',	'Логотип',	'Название',	'Email',	'Телефон',	'Сайт',	'Имя контактного лица', 'Тип работ', 'Технология'], axis= 1 , inplace= True)

In [ ]:
df_fabrics.tail(1)

,ID пользователя,ИНН,Вид изделий,Тип одежды,Сфера применения,По полу/возрасту,По сезону,Ценовые сегменты,Регион производства,Пошив образцов,...,Сырье,Нанесение,Оборудование,Штат,Упаковка,Условия оплаты,Дополнительные услуги,Сертификация,Загруженность,Наличии в реестре Минпромторга
871,404040,404040,Одежда,"легкая одежда, верхняя демисезонная одежда, до...","повседневная, образовательные учреждения, спор...","детская подростковая одежда, мужская, детск эк...",NaN,NaN,"цфо, юво",да,...,Давальческое,NaN,раскройный комплекс,NaN,"бирки, коробки",NaN,NaN,NaN,NaN,да


In [ ]:
df_fabrics_min=df_fabrics.copy()

In [ ]:
cols=df_fabrics_min.columns.values.tolist()


In [ ]:
df_fabrics_min = df_fabrics_min.astype(str)

In [ ]:
df_fabrics_min['fabrics_all_info'] = df_fabrics_min[['Вид изделий',	'Тип одежды',	'Сфера применения',	'По полу/возрасту',	'По сезону',	'Ценовые сегменты',	'Регион производства',	'Пошив образцов',	'Минимальная партия, шт.',	'Минимальная сумма заказа, руб','Заказчик должен предоставить','Конструирование',
                                                    'Сырье',	'Нанесение',	'Оборудование',	'Штат',	'Упаковка',	'Условия оплаты',	'Дополнительные услуги',	'Сертификация',	'Загруженность',	'Наличии в реестре Минпромторга']]. agg(' '. join , axis= 1 )

In [ ]:
df_fabrics_min.drop(['Вид изделий',	'Тип одежды',	'Сфера применения',	'По полу/возрасту',	'По сезону',	'Ценовые сегменты',	'Регион производства',	'Пошив образцов',	'Минимальная партия, шт.',	'Минимальная сумма заказа, руб','Заказчик должен предоставить','Конструирование',
                                                    'Сырье',	'Нанесение',	'Оборудование',	'Штат',	'Упаковка',	'Условия оплаты',	'Дополнительные услуги',	'Сертификация',	'Загруженность',	'Наличии в реестре Минпромторга'], axis=1, inplace= True)

In [ ]:
df_fabrics_min['fabrics_all_info']=df_fabrics_min['fabrics_all_info'].str.replace('nan', '')  #удалим nan из описания
df_fabrics_min['fabrics_all_info']=df_fabrics_min['fabrics_all_info'].str.replace(',', '')
df_fabrics_min['fabrics_all_info']=df_fabrics_min['fabrics_all_info'].str.lower()
df_fabrics_min.tail(4)

,ID пользователя,ИНН,fabrics_all_info
868,902,0,одежда джинсовые изделия легкая одежда алт...
869,188,3702664840,одежда головные уборы домашняя одежда легкая о...
870,505050,505050,одежда легкая одежда повседневная образователь...
871,404040,404040,одежда легкая одежда верхняя демисезонная одеж...


In [ ]:
df_fabrics_min['fabrics_all_info'][871]

'одежда легкая одежда верхняя демисезонная одежда домашняя одежда повседневная образовательные учреждения спортивная детская подростковая одежда мужская детск эконом   цфо юво да     давальческое  раскройный комплекс  бирки  коробки     да'

In [ ]:
df_fabrics_min['fabrics_all_info'][870]

'одежда легкая одежда повседневная образовательные учреждения детская подростковая одежда   цфо да     давальческое    бирки  пакеты     да'

##техзадание

In [ ]:
xl = pd.ExcelFile('ТЗ для Датасетов доп фабрики.xlsx')
df_orders = xl.parse('Техзадания v020424')
df_orders.tail(4)


,Сделано,Дата,Изображение,Комментарий к заказу,Вид изделия,Название,Тип одежды,Назначение,По полу и возрасту,Сезон,...,Плотность материала,Конструктивные особенности,Требования к оборудованию,Требование к штату,Упаковка,Условия оплаты,Дополнительные услуги,Сертификация,Срок поставки,Наличие в реестре Минпромторга
1229,1.0,2024-01-02 13:30:39,https://tpktrade.ru/wp-content/uploads/2024/01...,300 ШТ. на модель,Одежда,Боди из вискозы с эластаном,Нательное белье,Повседневная одежда,Женская одежда,Всесезон,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Обычная одежда, не требует обязательной сертиф...",2024-02-02,Да
1230,NaN,2024-01-02 13:30:36,https://tpktrade.ru/wp-content/uploads/2024/01...,Заказы на пошив трикотажной одежды из футера (...,Одежда,Мужская одежда,Трикотажная одежда,Спортивная одежда,Мужская одежда,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Обычная одежда, не требует обязательной сертиф...",2024-02-02,NaN
1231,1.0,2024-01-02 13:29:19,https://tpktrade.ru/wp-content/uploads/2024/01...,"Оборудование: двухиголка, петля прямая , пугов...",Одежда,Ветрозащитный костюм ВКПО,Спецодежда и СИЗ,Форменная одежда,Мужская одежда,NaN,...,NaN,NaN,"Оборудование: двухиголка, петля прямая , пугов...",К костюму еще идут подтяжки 2 ромбика по 1 пуг...,NaN,NaN,NaN,"Спецодежда и СИЗ, требующие обязательную серти...",2024-02-02,Нет
1232,1.0,2024-01-02 13:29:19,https://tpktrade.ru/wp-content/uploads/2024/01...,"шить детские брюки без подкладки, сырье даваль...",Одежда,Брюки без подкладки,Легкая одежда,Образовательные учреждения,Детская одежда,Всесезон,...,NaN,NaN,NaN,NaN,пакеты,NaN,NaN,"Обычная одежда, не требует обязательной сертиф...",2024-02-20,да


In [ ]:
column_names = df_orders.columns.tolist()


In [ ]:
df_orders.drop(['Сделано', 'Дата', 'Изображение', 'По полу и возрасту', 'Сезон', 'Ценовой сегмент', 'Регион поставки', 'Регионы производства', 'Образец', 'Пробная партия', 'Потенциал',
 'Кол-во цветов', 'Кол-во размеров', 'Количество моделей', 'Заказчик предоставит:', 'Конструирование', 'Обеспечение сырьем', 'Нанесение логотипа/принта', 'Виды нанесения', 'Цена за единицу', 'Плановый бюджет',
 'Описание сырья', 'Плотность материала', 'Конструктивные особенности', 'Требования к оборудованию', 'Требование к штату', 'Упаковка', 'Условия оплаты', 'Дополнительные услуги', 'Сертификация',
 'Срок поставки', 'Наличие в реестре Минпромторга'], axis= 1 , inplace= True)

In [ ]:
df_orders.head(1)

,Комментарий к заказу,Вид изделия,Название,Тип одежды,Назначение
0,NaN,Одежда,Фартуки,Легкая одежда,Униформа


In [ ]:
df_orders_min=df_orders.copy()

In [ ]:
cols_orders=df_orders_min.columns.values.tolist()
#cols_orders

In [ ]:
df_orders_min = df_orders_min.astype(str)

In [ ]:
df_orders_min['orders_all_info'] = df_orders_min[[ 'Вид изделия', 	'Название', 'Тип одежды', 'Назначение','Комментарий к заказу']]. agg(' '. join , axis= 1 )

In [ ]:
df_orders_min['orders_all_info']=df_orders_min['orders_all_info'].str.replace('nan', '')  #удалим nan из описания
df_orders_min['orders_all_info']=df_orders_min['orders_all_info'].str.replace(',', '')
df_orders_min['orders_all_info']=df_orders_min['orders_all_info'].str.lower()
df_orders_min.head(3)

,Комментарий к заказу,Вид изделия,Название,Тип одежды,Назначение,orders_all_info
0,nan,Одежда,Фартуки,Легкая одежда,Униформа,одежда фартуки легкая одежда униформа
1,ищу цех по пошиву поло 300 штук на вашем сырье...,Одежда,Футболки поло,Трикотажная одежда,Повседневная одежда,одежда футболки поло трикотажная одежда повсед...
2,nan,Одежда,Одежда для новорожденных,Нательное белье,Повседневная одежда,одежда одежда для новорожденных нательное бель...


In [ ]:
#выделим описания, по которым будем сравнивать заказы
text_to_compare = df_orders_min['orders_all_info'][[1232]]


In [ ]:
text_to_compare

1232    одежда брюки без подкладки легкая одежда образ...
Name: orders_all_info, dtype: object

##для заказов токенизация описания

In [ ]:
# импортируем класс для лемматизации
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

# создаём объект этого класса
lemmatizer = WordNetLemmatizer()

# и пустой список для слов после лемматизации
lemmatized = []

# проходимся по всем токенам
for token in text_to_compare:

    # применяем лемматизацию
    token_order = lemmatizer.lemmatize(token)

    # добавляем слово после лемматизации в список
    lemmatized.append(token)



[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
lemmatized

['одежда брюки без подкладки легкая одежда образовательные учреждения шить детские брюки без подкладки сырье давальческое цфо всесезонные  нужен крой и конструирование']

In [ ]:
#выделим основы слов в описании фабрик
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("russian")
orders_stemmed=[]  #сюда сформируем новый перечень описаний фабрик - только основы слов
for str in lemmatized:
 stemmed_words_orders = [stemmer.stem(word) for word in str.split()]
 row_stemmed_order = ' '.join(stemmed_words_orders)
 orders_stemmed.append(row_stemmed_order)




In [ ]:
orders_stemmed

['одежд брюк без подкладк легк одежд образовательн учрежден шит детск брюк без подкладк сыр давальческ цфо всесезон нуж кро и конструирован']

##для фабрик токенизация описания

In [ ]:
# импортируем класс для лемматизации
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

# создаём объект этого класса
lemmatizer = WordNetLemmatizer()

# и пустой список для слов после лемматизации
lemmatized = []

# проходимся по всем токенам
for token in df_fabrics_min['fabrics_all_info']:

    # применяем лемматизацию
    token = lemmatizer.lemmatize(token)

    # добавляем слово после лемматизации в список
    lemmatized.append(token)



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
lemmatized[:2]

['одежда верхняя демисезонная одежда верхняя зимняя одежда джинсовые изделия домашняя одежда изделия из меха костюмные изделия костюмные изделия легкая одежда майка-топ нательное белье трикотажная одежда футболка|2 more городская одежда деловая одежда офисная офисная деловая одежда повседневная повседневная городская одежда спортивная одежда торжественная одежда фирменная одежда/мерч/промо большие размеры детская одежда для беременных женская одежда мужская одежда подростковая одежда унисекс  средний средний плюс москва и мо нет 100 ед. на давальческом сырье от 300 ед. своей тканью   печать лекал на плоттере давальческое цифровая печать прямострочка конструктор на аутсорсинге 10 швей  аванс 50% / расчет 50% безналичная оплата безналичными счет без ндс  нет необходимо согласование не менее чем за 14 дней нет',
 'текстиль для дома и бизнеса одежда вязаная одежда джинсовые изделия домашняя одежда костюмные изделия костюмные изделия легкая одежда майка-топ нательное белье платочно-шарфовые

In [ ]:
#выделим основы слов в описании фабрик
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("russian")
fabrics_stemmed=[]  #сюда сформируем новый перечень описаний фабрик - только основы слов
for str in lemmatized:
 stemmed_words = [stemmer.stem(word) for word in str.split()]
 row_stemmed = ' '.join(stemmed_words)
 fabrics_stemmed.append(row_stemmed)




In [ ]:
#добавим сокращенные описания в датафрейм
df_fabrics_min["reduced_desript"]=fabrics_stemmed

In [ ]:
df_fabrics_min.head()

,ID пользователя,ИНН,fabrics_all_info,reduced_desript
0,8517,9731087878,одежда верхняя демисезонная одежда верхняя зим...,одежд верхн демисезон одежд верхн зимн одежд д...
1,8408,760414418671,текстиль для дома и бизнеса одежда вязаная оде...,текстил для дом и бизнес одежд вязан одежд джи...
2,8360,0,одежда домашняя одежда легкая одежда майка-топ...,одежд домашн одежд легк одежд майка-топ трикот...
3,8341,0,одежда домашняя одежда костюмные изделия легка...,одежд домашн одежд костюмн издел легк одежд ма...
4,8304,1308202110489,одежда костюмные изделия костюмные изделия лег...,одежд костюмн издел костюмн издел легк одежд м...


In [ ]:
#токенизируем все сокращенные тексты в нашем датасете
def tokenize_in_df(strin):
    try:
        return list(tokenize(strin, deacc=True,))
    except:
        return ""
df_fabrics_min["tokens"] = df_fabrics_min["reduced_desript"].apply(tokenize_in_df)

In [ ]:
#Создадим словарь слов, которые есть во всем нашем наборе текстов:
dictionary = corpora.Dictionary(df_fabrics_min["tokens"])
feature_cnt = len(dictionary.token2id)
#dictionary.token2id

In [ ]:
#Создаем корпус, превратив наши токенизированные тексты в векторы
corpus = [dictionary.doc2bow(text) for text in df_fabrics_min["tokens"]]

In [ ]:
# модель tf-idf
tfidf = models.TfidfModel(corpus)
index = similarities.SparseMatrixSimilarity(tfidf[corpus],num_features = feature_cnt)

In [ ]:
#векторы наших опорных текстов и получив значения их похожестей из матрицы.
for text in orders_stemmed:
    kw_vector = dictionary.doc2bow(tokenize(text))
    df_fabrics_min[text] = index[tfidf[kw_vector]]



In [ ]:
#можем избавиться от заведомо непохожих текстов, посчитав сумму весов и оставив тексты с самыми высокими суммами
df_fabrics_min["sum"] = 0
for text in orders_stemmed:
    df_fabrics_min["sum"] = df_fabrics_min["sum"]+df_fabrics_min[text]
    #df_fabrics_min["sum"]+df_fabrics_min[text]


In [ ]:
#Избавляться от лишних текстов можно обрезав по порогу суммы, или отсортировав по сумме и обрезав датасет по количеству текстов.

df_fabrics_min["sum"].value_counts(bins=10)

sum
(-0.001377, 0.0377]    745
(0.0377, 0.0753]        38
(0.0753, 0.113]         36
(0.113, 0.151]          30
(0.151, 0.188]          17
(0.188, 0.226]           4
(0.301, 0.339]           1
(0.339, 0.377]           1
(0.226, 0.264]           0
(0.264, 0.301]           0
Name: count, dtype: int64

In [ ]:
df_fabrics_results=df_fabrics_min[df_fabrics_min["sum"]>0.12]
df_fabrics_results = df_fabrics_results.sort_values(by='sum', ascending=False)
df_fabrics_results.to_excel("похожие тексты.xlsx")

In [ ]:
results = pd.read_excel('/content/похожие тексты.xlsx')
results

,Unnamed: 0,ID пользователя,ИНН,fabrics_all_info,reduced_desript,tokens,одежд брюк без подкладк легк одежд образовательн учрежден шит детск брюк без подкладк сыр давальческ цфо всесезон нуж кро и конструирован,sum
0,870,505050,505050,одежда легкая одежда повседневная образователь...,одежд легк одежд повседневн образовательн учре...,"['одежд', 'легк', 'одежд', 'повседневн', 'обра...",0.376601,0.376601
1,871,404040,404040,одежда легкая одежда верхняя демисезонная одеж...,одежд легк одежд верхн демисезон одежд домашн ...,"['одежд', 'легк', 'одежд', 'верхн', 'демисезон...",0.328832,0.328832
2,727,2359,666300682222,одежда вязаная одежда трикотажная одежда костю...,одежд вязан одежд трикотажн одежд костюмн изде...,"['одежд', 'вязан', 'одежд', 'трикотажн', 'одеж...",0.202063,0.202063
3,256,2373,0,одежда верхняя демисезонная одежда верхняя зим...,одежд верхн демисезон одежд верхн зимн одежд л...,"['одежд', 'верхн', 'демисезон', 'одежд', 'верх...",0.201167,0.201167
4,0,8517,9731087878,одежда верхняя демисезонная одежда верхняя зим...,одежд верхн демисезон одежд верхн зимн одежд д...,"['одежд', 'верхн', 'демисезон', 'одежд', 'верх...",0.194370,0.194370
5,666,1370,0,одежда трикотажная одежда легкая одежда домашн...,одежд трикотажн одежд легк одежд домашн одежд ...,"['одежд', 'трикотажн', 'одежд', 'легк', 'одежд...",0.189559,0.189559
6,108,4885,7725328306,одежда костюмные изделия верхняя демисезонная ...,одежд костюмн издел верхн демисезон одежд джин...,"['одежд', 'костюмн', 'издел', 'верхн', 'демисе...",0.183360,0.183360
7,95,6540,4029067069,аксессуары сумки одежда верхняя зимняя одежда ...,аксессуар сумк одежд верхн зимн одежд издел из...,"['аксессуар', 'сумк', 'одежд', 'верхн', 'зимн'...",0.177502,0.177502
8,660,1975,0,одежда костюмные изделия легкая одежда образов...,одежд костюмн издел легк одежд образовательн у...,"['одежд', 'костюмн', 'издел', 'легк', 'одежд',...",0.175346,0.175346
9,404,1979,0,аксессуары одежда верхняя демисезонная одежда ...,аксессуар одежд верхн демисезон одежд легк оде...,"['аксессуар', 'одежд', 'верхн', 'демисезон', '...",0.175128,0.175128
